In [6]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [7]:
# Carrega arquivo parquet
df = pd.read_parquet("DB/List-dependencies.parquet")

In [3]:
dfs = {}
graphs = {}
dfs["Geral"] = df
graphs["Geral"] = nx.from_pandas_edgelist(df, source="Project Name", target="Dependency Name", create_using=nx.DiGraph())
for platform in df["Platform"].unique():
    # cria um dataframe para cada plataforma "df["Platform"].unique()" e adiciona a lista de dataframes ao dicionário "dfs"
    dfs[platform] = df[df["Platform"] == platform]

    # Cria um grafo direcionado
    graphs[platform] = nx.from_pandas_edgelist(dfs[platform], source="Project Name", target="Dependency Name", create_using=nx.DiGraph())

In [ ]:
import pandas as pd
import os
import subprocess
from tqdm import tqdm

# --- 1. Preparação: Salvar DataFrames individuais ---

# Carrega o DataFrame principal uma única vez
df = pd.read_parquet("DB/List-dependencies.parquet")

# Define diretórios temporários e de saída
temp_data_dir = "temp_parquet_data"
output_dir = "analise_grafos"
os.makedirs(temp_data_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

print(f"Salvando arquivos de dados temporários em: '{os.path.abspath(temp_data_dir)}'")
print(f"Os relatórios finais serão salvos em: '{os.path.abspath(output_dir)}'")

# Cria uma lista de todas as plataformas para processar
platforms_to_process = ["Geral"] + list(df["Platform"].unique())

# Salva cada fatia do DataFrame como um arquivo Parquet separado
tasks_to_run = []
for name in platforms_to_process:
    file_path = os.path.join(temp_data_dir, f"{name}.parquet")
    if name == "Geral":
        df.to_parquet(file_path)
    else:
        # Filtra e salva o DataFrame da plataforma
        platform_df = df[df["Platform"] == name]
        platform_df.to_parquet(file_path)
    
    tasks_to_run.append({
        "name": name,
        "path": file_path
    })

print(f"{len(tasks_to_run)} arquivos de dados foram preparados.")


# --- 2. Orquestração: Lançar Processos Analisadores ---

# Limitar o número de processos simultâneos ainda é crucial
MAX_PROCESSES = 12
print(f"\nIniciando análise com no máximo {MAX_PROCESSES} processos paralelos...")

processes = []
task_queue = list(tasks_to_run) # Fila de tarefas

# Usamos a barra de progresso do TQDM
with tqdm(total=len(task_queue)) as pbar:
    while task_queue or processes:
        # Lança novos processos se houver espaço e tarefas
        while len(processes) < MAX_PROCESSES and task_queue:
            task = task_queue.pop(0)
            command = [
                "python", 
                "analyzer.py", 
                task["path"], 
                task["name"], 
                output_dir
            ]
            # Inicia o processo sem bloquear e esconde a janela do console no Windows
            proc = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, creationflags=subprocess.CREATE_NO_WINDOW)
            processes.append((proc, task["name"]))

        # Verifica os processos em execução
        for proc, name in processes:
            if proc.poll() is not None: # Processo terminou
                stdout, stderr = proc.communicate()
                if proc.returncode != 0:
                    print(f"\nERRO no processo para '{name}':")
                    print(stderr.decode('utf-8', errors='ignore'))
                else:
                    # Opcional: imprimir a saída padrão para ver o progresso
                    # print(stdout.decode('utf-8', errors='ignore'))
                    pass
                
                processes.remove((proc, name))
                pbar.update(1) # Atualiza a barra de progresso
        
        # Pequena pausa para não sobrecarregar a CPU com o loop de verificação
        time.sleep(0.5)

print("\n--- Análise de todos os grafos concluída! ---")
# Opcional: Limpar os arquivos temporários
# import shutil
# shutil.rmtree(temp_data_dir)
# print("Diretório de dados temporários removido.")

Salvando arquivos de dados temporários em: 'c:\Users\samuelcosta\Documents\cr-ufabc\temp_parquet_data'
Os relatórios finais serão salvos em: 'c:\Users\samuelcosta\Documents\cr-ufabc\analise_grafos'
19 arquivos de dados foram preparados.

Iniciando análise com no máximo 8 processos paralelos...


 95%|█████████▍| 18/19 [46:58<07:58, 478.22s/it]